In [ ]:
import numpy as np
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import os
import pytorch_lightning as pl
import pandas as pd
import torch
import torch.nn as nn
import torchmetrics

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from nvidia.dali import pipeline_def, Pipeline
from nvidia.dali.plugin.pytorch import DALIClassificationIterator
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from torch.optim import Adam
from tqdm import tqdm

from sklearn.metrics import(
    accuracy_score, 
    confusion_matrix, 
    ConfusionMatrixDisplay,
)

In [ ]:
media_pipe_preds = pd.read_csv("/mnt/durable/training_data/mediapipe-train.csv")
task_1_data = pd.read_csv("/mnt/durable/training_data/task1_pathml.csv")

In [ ]:
media_pipe_preds

In [ ]:
def extract_filename_label_frame(file_list):
    f = open(file_list, "r")
    total_files = f.readlines()
    f.close()

    labels = []
    video_ids = []
    frames = []
    for file in total_files:
        components = file.split(" ")

        # Get the file_name
        video_name = components[0].split("/")[-1]
        video_id = video_name.split(".")[0]

        label = components[1]

        start = float(components[2])
        end = float(components[3].strip())

        while start <= end:
            labels.append(label)
            video_ids.append(video_id)
            frames.append(start)
            start += 1

    df = pd.DataFrame(
        {
            "video_id": video_ids,
            "timestamp":frames,
            "label":labels,
        }  
    )
    return df

In [ ]:
testing_labels = extract_filename_label_frame(test_file_list)

In [ ]:
media_pipe_info = (
    media_pipe_preds
    .merge(testing_labels, how="left")
    .sort_values(["video_id", "timestamp"])
)

In [ ]:
media_pipe_info

In [ ]:
row = media_pipe_info.iloc[0]
label = row["label"]
start_index = 1

for i in range(start_index, media_pipe_info.shape[0]):
    row = media_pipe_info.iloc[i]
    if row.label != label:
        print(f"New Sequence from {start_index} to {i-1}, label: {label}")
        start_index = i
        label = row.label